In [1]:
from mosaik_heatpump.heatpump.Heat_Pump_Des import Heat_Pump_Des
import pandas as pd
import numpy as np
from tqdm import tqdm
from tespy.tools import logger
import logging
logger.define_logging(log_path=True, log_version=True,
                      screen_level=logging.ERROR,
                      file_level=logging.DEBUG)

'C:\\Users\\prana\\.tespy\\log_files\\tespy.log'

In [2]:
params_air = {
        'heat_source': 'air',
        'heat_source_T': 7,
        'cons_T': 35,
        'calc_mode': 'detailed'
    }

In [3]:
heat_pump_1 = Heat_Pump_Des(params_air)

In [12]:
inputs_air_1 = {'heat_source_T': -20, 'Q_Demand': 5000, 'cond_in_T': 25}

T_amb_list = np.arange(5, 20, 1)
cond_in_T_list = np.arange(45, 51, 1)

# T_amb_list = [7]
# cond_in_T_list = [47,48]

final_dict = {}
df = pd.DataFrame(index=T_amb_list, columns=cond_in_T_list)

In [13]:
i = 1
total = len(T_amb_list)*len(cond_in_T_list)
pbar = tqdm(total=total)
for T_amb in T_amb_list:
    final_dict[T_amb] = {}
    for cond_in_T in cond_in_T_list:
        heat_pump_1.heat_source_T = T_amb
        heat_pump_1.cond_in_T = cond_in_T
        heat_pump_1._etas_heatload_id()
        heat_pump_1._design_hp()
        heatload_list = np.arange(5000, heat_pump_1.heatload_des, 500)
        save_dict = {}
        for heatload in heatload_list:
            inputs = {'heat_source_T': T_amb, 'cond_in_T': cond_in_T, 'Q_Demand': heatload}
            heat_pump_1.step(inputs)
            save_dict[heatload] = {'cond_m': heat_pump_1.cond_m, 'COP': heat_pump_1.COP}

            if heat_pump_1.cond_m == 0 and heat_pump_1.COP == 0:
                print("Error for {} inputs".format(inputs))

        final_dict[T_amb][cond_in_T] = save_dict
        df.at[T_amb, cond_in_T] = save_dict
        pbar.update()

pbar.close()

  3%|▎         | 3/90 [00:19<09:25,  6.51s/it]

16:09:56-ERROR-Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting value for pressure.
-> given logarithmic temperature differences or kA-values for heat exchangers, 
-> support better starting values.
-> bad starting value for fuel mass flow of combustion chamber, provide small (near to zero, but not zero) starting value.
Error for {'heat_source_T': 5, 'cond_in_T': 48, 'Q_Demand': 5000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 48, 'Q_Demand': 5500.0} inputs
16:09:57-ERROR-Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting 

  4%|▍         | 4/90 [00:29<11:25,  7.97s/it]

Error for {'heat_source_T': 5, 'cond_in_T': 48, 'Q_Demand': 13500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 5000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 5500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 6500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 7000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 7500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 8000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 8500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 9000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 9500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 10000.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 10500.0} inputs
Error for {'heat_source_T': 5, 'cond_in_T': 49, 'Q_Demand': 11000.0} inputs
Error for {'heat_sour

  6%|▌         | 5/90 [00:48<16:28, 11.62s/it]

...

Error for {'heat_source_T': 19, 'cond_in_T': 49, 'Q_Demand': 5000.0} inputs
16:25:23-ERROR-Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting value for pressure.
-> given logarithmic temperature differences or kA-values for heat exchangers, 
-> support better starting values.
-> bad starting value for fuel mass flow of combustion chamber, provide small (near to zero, but not zero) starting value.
Error for {'heat_source_T': 19, 'cond_in_T': 49, 'Q_Demand': 5500.0} inputs
16:25:24-ERROR-Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate startin

 99%|█████████▉| 89/90 [15:57<00:12, 12.67s/it]

Error for {'heat_source_T': 19, 'cond_in_T': 49, 'Q_Demand': 18000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 5000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 6000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 6500.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 7000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 7500.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 8000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 8500.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 9000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 9500.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 10000.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 10500.0} inputs
Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 11000.0} inputs
Error fo

100%|██████████| 90/90 [16:25<00:00, 10.95s/it]

Error for {'heat_source_T': 19, 'cond_in_T': 50, 'Q_Demand': 18000.0} inputs


In [11]:
print(final_dict)
print(df)

{7: {47: {5000.0: {'cond_m': 0.6941216791831393, 'COP': 3.0582663963321286}, 5500.0: {'cond_m': 0.6941216791831393, 'COP': 3.010099708385097}, 6000.0: {'cond_m': 0.6941216791831393, 'COP': 2.963755801659261}, 6500.0: {'cond_m': 0.6941216791831393, 'COP': 2.9192835112142417}, 7000.0: {'cond_m': 0.6941216791831393, 'COP': 2.876491456657546}, 7500.0: {'cond_m': 0.6941216791831393, 'COP': 2.832381608047358}, 8000.0: {'cond_m': 0.6941216791831393, 'COP': 2.7891535723404397}, 8500.0: {'cond_m': 0.6941216791831393, 'COP': 2.7779551138416783}, 9000.0: {'cond_m': 0.6941216791831393, 'COP': 2.7874318297747944}, 9500.0: {'cond_m': 0.6941216791831393, 'COP': 2.794789717035178}, 10000.0: {'cond_m': 0.6941216791831393, 'COP': 2.7997941094660357}, 10500.0: {'cond_m': 0.6941216791831393, 'COP': 2.8020997262095197}, 11000.0: {'cond_m': 0.6941216791831393, 'COP': 2.8023041909044806}, 11500.0: {'cond_m': 0.6941216791831393, 'COP': 2.8001770530200782}, 12000.0: {'cond_m': 0.6941216791831393, 'COP': 2.7901